In [14]:
from google.colab import files
uploaded = files.upload()


Saving loan_data.csv to loan_data (2).csv


In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [17]:
df = pd.read_csv('loan_data.csv')
print(df.shape)
print(df.columns)


(45000, 14)
Index(['person_age', 'person_gender', 'person_education', 'person_income',
       'person_emp_exp', 'person_home_ownership', 'loan_amnt', 'loan_intent',
       'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length',
       'credit_score', 'previous_loan_defaults_on_file', 'loan_status'],
      dtype='object')


In [18]:
print(df.head())

   person_age person_gender person_education  person_income  person_emp_exp  \
0        22.0        female           Master        71948.0               0   
1        21.0        female      High School        12282.0               0   
2        25.0        female      High School        12438.0               3   
3        23.0        female         Bachelor        79753.0               0   
4        24.0          male           Master        66135.0               1   

  person_home_ownership  loan_amnt loan_intent  loan_int_rate  \
0                  RENT    35000.0    PERSONAL          16.02   
1                   OWN     1000.0   EDUCATION          11.14   
2              MORTGAGE     5500.0     MEDICAL          12.87   
3                  RENT    35000.0     MEDICAL          15.23   
4                  RENT    35000.0     MEDICAL          14.27   

   loan_percent_income  cb_person_cred_hist_length  credit_score  \
0                 0.49                         3.0           561  

In [19]:
print(df.isnull().sum())


person_age                        0
person_gender                     0
person_education                  0
person_income                     0
person_emp_exp                    0
person_home_ownership             0
loan_amnt                         0
loan_intent                       0
loan_int_rate                     0
loan_percent_income               0
cb_person_cred_hist_length        0
credit_score                      0
previous_loan_defaults_on_file    0
loan_status                       0
dtype: int64


In [34]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

categorical_cols = [
    'person_gender',
    'person_education',
    'person_home_ownership',
    'loan_intent',
    'previous_loan_defaults_on_file'
]

for col in categorical_cols:
    df[col] = le.fit_transform(df[col])
df[categorical_cols].head()



,person_gender,person_education,person_home_ownership,loan_intent,previous_loan_defaults_on_file
0,0,4,3,4,0
1,0,3,2,1,1
2,0,3,0,3,0
3,0,1,3,3,0
4,1,4,3,3,0


In [32]:
X = df.drop('loan_status', axis=1)
y = df['loan_status']
print(X.shape)
print(y.shape)



(45000, 13)
(45000,)


In [31]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print(X_scaled[:5])



[[-0.95353824 -1.11006918  1.52526448 -0.10408961 -0.89228413  0.90482152
   4.02495343  0.85291945  1.6830388   4.01639414 -0.73910854 -1.41981408
  -1.01603973]
 [-1.11896309 -1.11006918  0.84919205 -0.84600467 -0.89228413  0.21060873
  -1.35920933 -0.8829042   0.04478157 -0.6848294  -0.99686317 -2.5499748
   0.98421348]
 [-0.45726369 -1.11006918  0.84919205 -0.84406489 -0.3975175  -1.17781684
  -0.64659956  0.27431157  0.62555719  3.4430742  -0.73910854  0.04741211
  -1.01603973]
 [-0.78811339 -1.11006918 -0.50295282 -0.00703857 -0.89228413  0.90482152
   4.02495343  0.27431157  1.41782913  3.4430742  -0.99686317  0.84050735
  -1.01603973]
 [-0.62268854  0.90084476  1.52526448 -0.17637118 -0.72736192  0.90482152
   4.02495343  0.27431157  1.09554902  4.4750501  -0.48135391 -0.92412955
  -1.01603973]]


In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)



(36000, 13)
(9000, 13)
(36000,)
(9000,)


In [25]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=200,
    random_state=42
)

model.fit(X_train, y_train)


RandomForestClassifier(n_estimators=200, random_state=42)

In [26]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.9274444444444444
              precision    recall  f1-score   support

           0       0.94      0.97      0.95      6990
           1       0.89      0.77      0.83      2010

    accuracy                           0.93      9000
   macro avg       0.91      0.87      0.89      9000
weighted avg       0.93      0.93      0.93      9000



In [27]:
!pip install gradio


In [28]:
import gradio as gr

def predict_loan(
    age, gender, education, income, emp_exp,
    home_ownership, loan_amount, loan_intent,
    loan_int_rate, loan_percent_income,
    credit_hist_length, credit_score,
    prev_defaults
):
    input_data = np.array([[
        age, gender, education, income, emp_exp,
        home_ownership, loan_amount, loan_intent,
        loan_int_rate, loan_percent_income,
        credit_hist_length, credit_score,
        prev_defaults
    ]])

    input_scaled = scaler.transform(input_data)
    prediction = model.predict(input_scaled)

    return "Approved" if prediction[0] == 1 else "Rejected"


In [29]:
interface = gr.Interface(
    fn=predict_loan,
    inputs=[
        gr.Number(label="Age"),
        gr.Number(label="Gender (Encoded)"),
        gr.Number(label="Education (Encoded)"),
        gr.Number(label="Annual Income"),
        gr.Number(label="Employment Experience (Years)"),
        gr.Number(label="Home Ownership (Encoded)"),
        gr.Number(label="Loan Amount"),
        gr.Number(label="Loan Intent (Encoded)"),
        gr.Number(label="Loan Interest Rate"),
        gr.Number(label="Loan Percent Income"),
        gr.Number(label="Credit History Length"),
        gr.Number(label="Credit Score"),
        gr.Number(label="Previous Loan Defaults (Encoded)")
    ],
    outputs="text",
    title="Loan Approval Prediction System",
    description="Enter applicant details to predict loan approval status"
)

interface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ec6eb945fdaa9afd25.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
